In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import pandas as pd
import time
from datetime import datetime

In [2]:
usuarios_datos=pd.DataFrame(pd.read_excel(r'path to csv with user data'))
usu=str(usuarios_datos['usuario'][0])
clave=str(usuarios_datos['clave'][0])
mat=str(usuarios_datos['matricula'][0])
cuit=str(usuarios_datos['cuit'][0])



In [3]:
prestaciones=pd.DataFrame(pd.read_excel(r'path to excel with data to upload'))


In [6]:
#acceso y login
driver = webdriver.Chrome(service=Service(executable_path=ChromeDriverManager().install()))
#driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://menu.traditum.com/View/Login.aspx")
driver.find_element(By.NAME,'txtUsuario').send_keys(usu)#cambiar a un objeto el usuario y contrasena
driver.find_element(By.NAME, 'txtPassword').send_keys(clave)
driver.find_element(By.ID,'btnLogin').click()
#seleccion de medife y solicitud de autorizacion
driver.find_element(By.CLASS_NAME,'navbar-menu').click()#click en menu
hoverable = driver.find_element(By.ID, "rptFinanciadoras_lblFinanciadora_0")
ActionChains(driver)\
    .move_to_element(hoverable).perform()
driver.find_element(By.LINK_TEXT,'Solicitud de autorización').click()
#carga de datos de prestador
driver.switch_to.frame('myIframe') 
driver.find_element(By.ID,'bodyContent_PRD_EFE_txtNroIDPrestador').send_keys(mat)
driver.find_element(By.ID,'bodyContent_PRD_PRE_txtNroIDPrestador').send_keys(cuit) 
#cargo paciente
#aca empezaria el for loop
for i in prestaciones.index:
    time.sleep(3)
    afi=WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID,'bodyContent_PID_txtNroIDAfiliado')))
    if afi.get_attribute('value')=='':
        afi.send_keys(str(prestaciones['n af'][i]))
    else:
        time.sleep(3)
        driver.find_element(By.ID,'bodyContent_PID_btnNroIDAfiliado_Clean').click()
        afi.send_keys(str(prestaciones['n af'][i]))
    #probar set wait el boton abajo esta inactivo
    #cargo tipo de prestacion internada vs programada
    prog_int=Select(driver.find_element(By.ID,'bodyContent_PV1Internacion_selTipoAdmision'))
    prog_int.select_by_value('P')
    #codigo de diagnostico
    diag=driver.find_element(By.ID,'bodyContent_DG1_txtNroIDDiagnostico').get_attribute('value')
    if diag=='':
        driver.find_element(By.ID,'bodyContent_DG1_txtNroIDDiagnostico').send_keys('I10')
    #selector de practicas
    tip_prac=Select(driver.find_element(By.ID,'bodyContent_PRE_selSistemaCodificacion'))
    tip_prac.select_by_value('1')
    #Codigo de prestacion
    prest= driver.find_element(By.ID,'bodyContent_PRE_txtCodigoPrestacion')
    if prest=='':
        prest.send_keys(str(prestaciones['codigo'][i]))
    else:
        prest.clear()
        prest.send_keys(str(prestaciones['codigo'][i]))
    #seleccionar cantidad de prestaciones, si no hay nada setear a uno, si uno no tocar
    cant=driver.find_element(By.ID,'bodyContent_PRE_txtCantidadTratamientoSolicitados').get_attribute('value')
    if cant == '':
        driver.find_element(By.ID,'bodyContent_PRE_txtCantidadTratamientoSolicitados').send_keys('1')
    aceptar=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "bodyContent_PRE_btnTablaPrestaciones_Add")))
    time.sleep(2)
aceptar.click()
time.sleep(2)
driver.find_element(By.ID,'bodyContent_btnAceptar').click()
driver.find_element(By.ID,'bodyContent_btnAceptar').click()
    #chequear si con el dos hace dos click
#descargar comprobante
driver.find_element(By.ID,'bodyContent_btnExportar').click()

AttributeError: 'NoneType' object has no attribute 'click'